In [ ]:
link = 'https://www.pse.pl/dane-systemowe/funkcjonowanie-rb/raporty-dobowe-z-funkcjonowania-rb/podstawowe-wskazniki-cenowe-i-kosztowe/rynkowa-cena-energii-elektrycznej-rce'
params_sel = 'span[class=nav-item-label]'
date_input_sel = 'input[name=data]'
confirm_sel = 'button[class*=apply]'
table_sel = 'tbody[id*=yui_patched]'

In [ ]:
import contextlib
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import TimeoutException, NoSuchWindowException
from typing import Union
from bs4 import BeautifulSoup as Bs
import pandas as pd
from datetime import datetime, timedelta
from driver_factory import DriverFactory
import time


class LimitExceededException(Exception):
    def __init__(self, date: str, *args: object) -> None:
        mes = f'limit exceeded at {date}'
        super().__init__(mes, *args)


DATE = 'Date'
PRICE = 'Price'
HOUR = 'Hour'
TIMEOUT = 10
WAIT = 0.3

def set_driver(headless: bool = True, driver = None):
    if driver is None:
        driver = DriverFactory.get_driver(headless=headless)    
    driver.get(link)
    return driver

def read_table(table_html: Bs) -> pd.DataFrame:
    df = pd.read_html(table_html)[0]
    converters = {df.columns[1]: lambda x: str(x.replace(u'\xa0', u''))}
    df = pd.read_html(table_html,
                      converters=converters,
                      decimal=',',
                      thousands='.')[0].dropna()
    df.columns = [HOUR, PRICE]
    return df

def transform_table(data: pd.DataFrame, date: str) -> pd.DataFrame:
    df = data.copy()
    df[PRICE] = pd.to_numeric(df[PRICE])
    df[DATE] = df[HOUR].apply(lambda x: date + timedelta(hours=x))
    df = df.drop(columns=HOUR)
    return df

def get_element(driver, selector: str, timeout: Union[int, float] = TIMEOUT) -> WebElement:
    element = WebDriverWait(driver=driver,
                            timeout=timeout).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
    return element


def scrap_prices(dates: pd.DatetimeIndex, headless: bool = True):
    driver = set_driver(headless=headless)
    prices = pd.DataFrame()
    with contextlib.suppress(
        ValueError,
        TimeoutException,
        NoSuchWindowException,
        KeyboardInterrupt
    ):
        for date in dates:
            day_df = get_prices(date=date, driver=driver)
            prices = pd.concat([prices, day_df], ignore_index=True)
            
    prices = prices[[DATE, PRICE]].sort_values(DATE)
    return prices

def click_params(driver):
    params = get_element(driver=driver, selector=params_sel)
    params.click()
    time.sleep(WAIT)
    
def send_keys(driver, date: str):
    date_input = get_element(driver=driver, selector=date_input_sel)
    date_input.clear()
    date_input.send_keys(datetime.strftime(date, '%Y-%m-%d'))
    
def confirm(driver):
    confirm = get_element(driver=driver, selector=confirm_sel)
    confirm.click()
    time.sleep(WAIT)

def get_table_html(driver) -> str:
    table = get_element(driver=driver, selector=table_sel)
    table_html = table.get_attribute('innerHTML')
    return table_html
    
def get_prices(date: datetime, driver) -> pd.DataFrame:
    time.sleep(WAIT)
    click_params(driver=driver)
    send_keys(driver=driver, date=date)
    confirm(driver=driver)
    table_html = get_table_html(driver=driver)
    df = read_table(table_html=table_html)
    df = transform_table(data=df, date=date)
    return df
    
# now = datetime.now()
# r = pd.date_range('2018-01-01', now)
# pr = scrap_prices(dates=r, headless=False)

In [ ]:
driver = DriverFactory.get_driver(headless=False) 
driver.get(link)

In [ ]:
def get_element(driver, selector: str, timeout: Union[int, float] = TIMEOUT) -> WebElement:
    element = WebDriverWait(driver=driver,
                            timeout=timeout).until(
    EC.visibility_of_any_elements_located((By.CSS_SELECTOR, selector)))
    return element

get_element(driver, 'span[class=nav-item-label]')[-1].click()

In [ ]:
from_, to = get_element(driver, 'input[id*=_VisioToolbar')

In [ ]:
from_.clear()
from_.send_keys('2022-12-01')
to.clear()
to.send_keys('2022-12-10')

In [ ]:
get_element(driver, 'button[class*=ui-datepicker-close]')[0].click()

In [ ]:
confirm = get_element(driver, "a[title='Eksport do CSV']")[-1].click()

In [ ]:
pr.set_index(DATE).plot()

In [ ]:
import contextlib
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import TimeoutException, NoSuchWindowException, NoSuchElementException
from typing import Union
from bs4 import BeautifulSoup as Bs
import pandas as pd
from datetime import datetime, timedelta
from driver_factory import DriverFactory
import time
import requests

main_page = 'https://www.pogodajutro.com/europe/poland?page=past-weather'


def set_up(headless: bool = True):
    driver = DriverFactory.get_driver(headless=headless)    
    driver.get(main_page)
    time.sleep(2)
    consent = driver.find_element(By.CSS_SELECTOR, "button[class='fc-button fc-cta-consent fc-primary-button']")
    consent.click()
    time.sleep(1)
    return driver

def build_df(driver, dates):
    weather = pd.DataFrame()
    for date in dates:
        print(date)
        link = f'{main_page}#day={date.day}&month={date.month}'
        driver.get(link)
        for year in (2019, 2020, 2021, 2022):
            try:
                time.sleep(1)
                year_tab = driver.find_element(By.CSS_SELECTOR, f"button[data-year='{year}']")
                driver.execute_script("arguments[0].click();", year_tab)
            except NoSuchElementException:
                print(f'No data for {year} {date.month} {date.day}')
                print(link)
                continue
            html = driver.page_source
            df = pd.read_html(html)[2]
            xd = pd.melt(df, id_vars='Unnamed: 0', var_name='Hour')
            xd = pd.pivot(xd, index='Hour', columns='Unnamed: 0', values='value').reset_index()
            xd['Year'] = year
            xd['Month'] = date.month
            xd['Day'] = date.day
            xd['Time'] = pd.to_datetime(xd[['Year', 'Month', 'Day', 'Hour']].astype(str).agg('-'.join, axis=1))
            xd = xd.drop(columns=['Year', 'Month', 'Day', 'Hour'])
            weather = pd.concat([weather, xd], ignore_index=True)
    return weather

driver = set_up(headless=True)
dates = pd.date_range('01-01-2012', '12-31-2012', freq='D')
assert len(dates) == 366
df = build_df(driver, dates)


In [ ]:
df.to_csv('weather_data.csv', index=False)

In [30]:
import pandas as pd
data = pd.read_csv('weather_data.csv', parse_dates=['Time'], index_col="Time")

In [31]:
import json

with open('data_description.json', 'r', encoding='utf-8') as f:
    desc = json.load(f)

In [24]:
data.rename(columns=desc['Renames']['ENG'], inplace=True)

In [10]:
units = ['%', '°  C', 'Km/h', ' Km', ' mm', '°']
replace = {unit: '' for unit in units}
replace[','] = '.'
print(replace)
data.replace(replace, regex=True, inplace=True)

for col in data.select_dtypes(object).columns:
    data[col] = data[col].str.strip(')').str.lstrip('(')


{'%': '', '°  C': '', 'Km/h': '', ' Km': '', ' mm': '', '°': '', ',': '.'}


In [11]:
numeric = data.columns.drop(['Kierunek wiatru', 'Pogoda', 'Time'])
data[numeric] = data[numeric].astype(float)

In [ ]:
data = pd.read_csv('weather_data.csv', parse_dates=['Time'])


In [42]:
data.to_csv('weather_data.csv')

In [73]:
import contextlib
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import TimeoutException, NoSuchWindowException, NoSuchElementException
from typing import Union
from bs4 import BeautifulSoup as Bs
import pandas as pd
from datetime import datetime, timedelta
from driver_factory import DriverFactory
import time
import requests

# Eurosuper 95
# other trype oil_98
main_page = 'https://www.lotos.pl/145/type,oil_95/dla_biznesu/hurtowe_ceny_paliw/archiwum_cen_paliw'


driver = DriverFactory.get_driver(headless=False)
driver.get(main_page)
time.sleep(2)
html = driver.page_source

driver: C:\Users\wojte\.wdm\drivers\chromedriver\win32\108.0.5359\chromedriver.exe
driver_version: 108.0.5359.71
browser_version: 108.0.5359.125


In [74]:
df = pd.read_html(html)[0]
df


,Data zmiany,Cena,Akcyza,Opłata paliwowa
0,2022-12-24,"5 838,00","1 413,00",15261
1,2022-12-23,"5 828,00","1 413,00",15261
2,2022-12-22,"5 828,00","1 413,00",15261
3,2022-12-21,"5 836,00","1 413,00",15261
4,2022-12-20,"5 773,00","1 413,00",15261
...,...,...,...,...
4598,2004-01-21,"2 650,00","1 514,00",7928
4599,2004-01-16,"2 615,00","1 514,00",7928
4600,2004-01-13,"2 615,00","1 514,00",7928
4601,2004-01-08,"2 580,00","1 514,00",7928


In [75]:
df = df.rename(columns={"Data zmiany": "Data"})
df = df.drop(columns=['Akcyza', 'Opłata paliwowa'])
df['Data'] = pd.to_datetime(df['Data'])
df['Cena'] = pd.to_numeric(df['Cena'].replace({' ': '', ',': '.'}, regex=True))
df


,Data,Cena
0,2022-12-24,5838.0
1,2022-12-23,5828.0
2,2022-12-22,5828.0
3,2022-12-21,5836.0
4,2022-12-20,5773.0
...,...,...
4598,2004-01-21,2650.0
4599,2004-01-16,2615.0
4600,2004-01-13,2615.0
4601,2004-01-08,2580.0


In [76]:
df.set_index('Data').to_csv('oil_prices.csv')
